# Дополнительные задачи ответы в конце

## Задача 1 Что будет на выходе? 

In [4]:
recs = [(1, 2, 3), (1, 2), (3, 4), (0, 1, 2, 3, 4)]
s = 0
for a, *b in recs:
    s += sum(b)
#print(s)

## Задача 2 Что будет на выходе? 

In [6]:
x = [[0]]
x = x + x*2
x[0].append(1)
#print(x)

# Вопросы
- 1 Как работает логистическая регрессия?
- 2 Почему оптимизация логлоса приводит к оценке вероятностей?
- 3 Как работает градиентный бустинг? А в задаче классификации? 
- 4 Вопросы про глубину и количество деревьев в GBM и RF? 
- 5 Чем хорош и плох ROC-AUC? Насколько он уместен с несбалансированными выборками? А в задачах Ранжирования? 
- 6 Как оценивать статзначимость в А/В?


Ответ дается в краткой форме + ссылка на дополнительные материалы

# 4 Глубина и количество деревьев в GBM и RF

## 4.1 Gradient Boosting Model

Градиентный Бустинг - это метод машинного обучения, используемый, в частности, в задачах регрессии и классификации. Он дает модель предсказания **в виде ансамбля слабых моделей предсказания**, которые обычно являются деревьями решений. Когда дерево решений является слабым обучаемым, результирующий алгоритм называется gradient-boosted trees; он обычно превосходит random forest. Модель gradient-boosted trees строится поэтапно, как и в других методах бустинга, но она обобщает другие методы, позволяя оптимизировать произвольную дифференцируемую функцию потерь.

### Кратко: Много (тысячи) деревьев небольшой глубины (до 10). 

### Ниже приведены отрывки из [Градиентный бустинг - Учебник по ML от ШАД ](https://ml-handbook.ru/chapters/grad_boost/intro)


Типичный градиентный бустинг имеет в составе **несколько тысяч деревьев решений**, которые необходимо строить последовательно. Построение решающего дерева на выборках типичного размера и современном железе, даже с учетом всех оптимизаций, требует небольшого, но всё-таки заметного времени (0.1-1c), которое для всего ансамбля превратится в десятки минут. Это не так быстро, как обучение линейных моделей, но всё-таки значительно быстрее, чем обучение типичных нейросетей.

### Темп обучения (learning rate)

Обучение композиции с помощью градиентного бустинга может привести к переобучению, если базовые алгоритмы слишком сложные. Например, **если сделать решающие деревья слишком глубокими (более 10 уровней)**, то при обучении бустинга ошибка на обучающей выборке даже при довольно скромном K может приблизиться к нулю, то есть предсказание будет почти идеальным, но на тестовой выборке всё будет плохо.

Существует два решения этой проблемы. Во-первых, необходимо упростить базовую модель, **уменьшив глубину дерева** (либо примерив какие-либо ещё техники регуляризации). Во-вторых, мы можем ввести параметр, называемый темпом обучения (learning rate). Присутствие этого параметра означает, что каждый базовый алгоритм вносит относительно небольшой вклад в итоговое решение.


### Feature importance

Отдельные деревья решений можно легко интерпретировать, просто визуализируя их структуру. Но в модели градиентного бустинга содержатся сотни деревьев, и поэтому её нелегко интерпретировать путем визуализации входящих в неё деревьев. При этом хотелось бы, как минимум, понимать, какие именно признаки в данных оказывают наибольшее влияние на предсказание композиции.

Можно сделать следующее наблюдение: признаки, используемые в верхней части дерева, влияют на окончательное предсказание для большей доли обучающих объектов, чем признаки, попавшие на более глубокие уровни. Таким образом, ожидаемая доля обучающих объектов, для которых происходило ветвление по данному признаку, может быть использована в качестве оценки его относительной важности для итогового предсказания. Усредняя полученные оценки важности признаков по всем решающим деревьям из ансамбля, можно уменьшить дисперсию такой оценки и использовать ее для отбора признаков.


### Где используется

Везде :) На сегодня градиентный бустинг – это, фактически, один из двух подходов, которые используются на практике (второй – это нейронные сети, конечно). Он формально слабее и менее гибок, чем сети, но выигрывает в простоте настройки темпа обучения и применения, размере и интерпретируемости модели.

Во многих компаниях, так или иначе связанных с ML, он используется для всех задач, которые не связаны с однородными данными (картинками, текстами, etc). Типичный поисковый запрос в Яндексе, выбор отеля на Booking.com или сериала на вечер в Netflix задействует несколько десятков моделей GBDT.

Впрочем, в будущем можно ожидать плавного исчезновения этого подхода, так как улучшение архитектур глубинного обучения и дальнейшее развитие железа нивелирует его преимущество по сравнению с нейросетями.

## 4.2 Random Forest


Случайные леса или случайные леса решений - это метод ансамблевого обучения (**Bagging**) для классификации, регрессии и других задач, который работает путем построения множества деревьев решений во время обучения.
-  Для задач классификации выходным результатом случайного леса является класс, выбранный большинством деревьев.
-  Для задач регрессии возвращается средний или усредненный прогноз отдельных деревьев. 

Случайные леса решений исправляют привычку деревьев решений чрезмерно подстраиваться под обучающий набор. Случайные леса в целом превосходят деревья решений, но их точность ниже, чем у деревьев с градиентным усилением. Однако характеристики данных могут влиять на их производительность.

### Кратко: Регулируем число деревьев на обучаемой выборке, деревья строим глубокие.


### Ниже приведены отрывки из [Ансамбли в машинном обучении - Учебник по ML от ШАД ](https://ml-handbook.ru/chapters/ensembles/intro#random-forest)

### Какая должна быть глубина деревьев в случайном лесе?
Ошибка модели (на которую мы можем повлиять) состоит из смещения и разброса. Разброс мы уменьшаем с помощью процедуры бэггинга. На смещение бэггинг не влияет, а хочется, чтобы у леса оно было небольшим. Поэтому смещение должно быть небольшим у самих деревьев, из которых строится ансамбль. У неглубоких деревьев малое число параметров, то есть дерево способно запомнить только верхнеуровневые статистики обучающей подвыборки. Они во всех подвыборках будут похожи, но будут не очень подробно описывать целевую зависимость. Поэтому при изменении обучающей подвыборки предсказание на тестовом объекте будет стабильным, но не точным (низкая дисперсия, высокое смещение). Наоборот, у глубоких деревьев нет проблем запомнить подвыборку подробно. Поэтому предсказание на тестовом объекте будет сильнее меняться в зависимости от обучающей подвыборки, зато в среднем будет близко к истине (высокая дисперсия, низкое смещение). 

**Вывод: используем глубокие деревья.**

### Сколько должно быть деревьев в случайном лесе?
Выше было показано, что увеличение числа элементарных алгоритмов в ансамбле не меняет смещения и уменьшает разброс. Так как число признаков и варианты подвыборок, на которых строятся деревья в случайном лесе, ограничены, уменьшать разброс до бесконечности не получится. 

**Вывод 1: Поэтому имеет смысл построить график ошибки от числа деревьев и ограничить размер леса в тот момент, когда ошибка перестанет значимо уменьшаться.**

Вторым практическим ограничением на количество деревьев может быть время работы ансамбля. Однако есть положительное свойство случайного леса: случайный лес можно строить и применять параллельно, что сокращает время работы, если у нас есть несколько процессоров. Но процессоров, скорее всего, всё же сильно меньше числа деревьев, а сами деревья обычно глубокие. 

**Вывод 2: Поэтому на большом числе деревьев Random Forest может работать дольше желаемого и количество деревьев можно сократить, немного пожертвовав качеством.**

# 5 ROC-AUC.  Плюсы и минусы. Несбалансированные выборки. Задача ранжирования.

## 5.1 ROC-AUC 

Кратко
ROC - кривая характеризующая модели классификации (при изменении порога чувствительности модели threshold), которые возвращают "оценку" = вероятность принадлежности классу {0,1}.

ПЛЮСЫ: позволяет визуально сравнить эффективность разных моделей и охватывают несколько аспектов классификации ( ложно положительные и ложно отрицательные результаты)

более подробнее надо дать определение: 
 - порог чувствительности модели threshold (Если значение в модели = вероятность > threshold, то классифицируем как 1 = ожирение, спам, мошенник)
 - доля истинно положительных результатов (True Positive Rate) TPR = TP/(TP+FN)
 - доля ложно положительных результатов (False Positive Rate)  FRP = FP/(FP+TN) 

AUC - area under curve, "величина" от 0 до 1 позволяющая, наиболее быстро сравнивать ROC кривые разных моделей. 
- Когда AUC выше 0.5 значит модель предсказывает лучше, чем предсказывать случайным выбором (AUC=0.5).
- Когда AUC ниже 0.5 значит, с моделью что-то не так.


 Источники:
 -  [StatQuest](https://www.youtube.com/watch?v=4jRBRDbJemM&t=19s)
 - [Александр Дьяконов: AUC ROC (площадь под кривой ошибок)](https://dyakonov.org/2017/07/28/auc-roc-%D0%BF%D0%BB%D0%BE%D1%89%D0%B0%D0%B4%D1%8C-%D0%BF%D0%BE%D0%B4-%D0%BA%D1%80%D0%B8%D0%B2%D0%BE%D0%B9-%D0%BE%D1%88%D0%B8%D0%B1%D0%BE%D0%BA/)


## 5.2 Несбалансированные выборки

Кратко: 
Задачи с дисбалансом чаще всего возникают, когда какой-то из классов соответствует очень редко наблюдаемым или диагностируемым явлениям (дефолт, поломка, редкая болезнь, мошенничество и т.п.)

Причины дисбалансов бывают разные и им соответствуют разные механики решения. 

Если у нас обычная бинарная задача классификации, пропорции классов не меняются со временем и процент объектов класса 1 – от 2% до 10%. Эта ситуация соответствует, например, задаче банковского скоринга в стабильной экономической обстановке.

Ключевой проблемой в данном случае является выбор порога чувствительности модели threshold. 


### Дополнительно:
- [Александра Дьяконова: Дисбаланс классов](https://dyakonov.org/2021/05/27/imbalance/)
- [Jason Brownlee: ROC Curves and Precision-Recall Curves for Imbalanced Classification](https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-imbalanced-classification/)

## 5.3 Задача ранжирования

Часто встречаются задачи, в которых целевой признак по-прежнему бинарный,
но при этом необходимо ранжировать объекты, а не просто предсказывать их класс.
Например, в задаче предсказания реакции клиента можно выдавать сортированный
список, чтобы оператор мог в первую очередь позвонить клиентам с наибольшей
вероятностью положительного отклика.
Поскольку многие алгоритмы возвращают
вещественный ответ b(x), который затем бинаризуется по порогу t, то можно просто
сортировать объекты по значению b(x). 

Критерий AUC-ROC имеет большое число интерпретаций — например, его можно использовать как вероятности того, что случайно выбранный положительный объект окажется выше случайно выбранного отрицательного объекта в ранжированном списке, порожденном b(x).

### Источник:
[Евгений Соколов: Семинары по выбору моделей](http://www.machinelearning.ru/wiki/images/1/1c/sem06_metrics.pdf)


# 6 Статистическая значимость в А/В тестах. Как оценивать?


У Вас сейчас применяются параметры А, а вы хотите понять, а что если попробовать параметры В?
Вы провели замеры и получили:
 - при параметрах A: 11 "успеха" из 100 "попыток"
 - при параметрах В: 13 "успеха" из 100 "попыток"

[Evan Miller calculator](https://www.evanmiller.org/ab-testing/chi-squared.html#!11/100;13/100@95)

А если бы Вы получили:
 - при параметрах A: 110 "успеха" из 1000 "попыток"
 - при параметрах В: 130 "успеха" из 1000 "попыток"

А если бы Вы получили:
 - при параметрах A: 1100 "успеха" из 10000 "попыток"
 - при параметрах В: 1300 "успеха" из 10000 "попыток"

[Evan Miller calculator](https://www.evanmiller.org/ab-testing/chi-squared.html#!1100/10000;1300/10000@95)


Вопрос: В каком из экспериментов можно утверждать, что теперь надо перейти на параметры В?

Ответ: Мы должны быть уверены, что результаты эксперимента не случайны, то есть, что параметры B будут приносить больше "успеха", чем параметры А "регулярно".

Для этого вводят и используют **Уровень статистической значимости** (α). Это уровень риска, который вы принимаете при ошибках первого рода (отклонение нулевой гипотезы, если она верна), обычно α = 0.05. Это означает, что в 5% случаев вы будете обнаруживать разницу между A и B, которая на самом деле обусловлена случайностью. Чем ниже выбранный вами уровень значимости, тем ниже риск того, что вы обнаружите разницу, вызванную случайностью.


На результат и обоснованность экспериментов влияют:
 - масштабные изменения в среде (новостные события, природные катаклизмы, распродажа у конкурентов),
 - предопределенность экспериментатора (bias) и ошибки проведения эксперимента.


Используем слишком высокий - тесты возможных улучшений будут проваливаться, хотя улучшения на самом деле есть. Используем слишком низкий - часто будем получать "подтверждения" ложных улучшений.

Независимо от выбранного уровня значимости, принимая решения по результатам AB-тестов, время от времени мы будем ошибаться и наносить ущерб бизнесу. Выбирая уровень статистической значимости тестов, мы можем ограничить количество ошибок и балансировать между пользой от оправданно успешных экспериментов и ущербом от ошибочно успешных.

### Дополнительно:

[academy.yandex.ru: Как провести A/B-тестирование: 6 простых шагов](https://academy.yandex.ru/posts/kak-provesti-a-b-testirovanie-6-prostykh-shagov),

[Habr: Как выбрать уровень статистической значимости для AB-теста и как интерпретировать результат](https://habr.com/ru/post/554194/),

[Product Analyst из Яндекс: "Разбираемся с нуля в А/Б-тестах"](https://productstar.ru/ab-test) или [видео](https://youtu.be/F3yGiFY9Nmo?t=3961).



## Ответы на задачи 

In [1]:
recs = [(1, 2, 3), (1, 2), (3, 4), (0, 1, 2, 3, 4)]
s = 0
for a, *b in recs:
    print('a = ',a)
    print('b = ',b)
    print('--------------')
    s += sum(b)
print('s = ',s)

a =  1
b =  [2, 3]
--------------
a =  1
b =  [2]
--------------
a =  3
b =  [4]
--------------
a =  0
b =  [1, 2, 3, 4]
--------------
s =  21


In [10]:
x = [[0]]
print('x = ',x)
print('x*2 = ',x*2)
x = x + x*2
print('after x = x + x*2\nx = ',x)
print('x[0] = ', x[0])
x[0].append(1)
print('after x[0].append(1)\n',x)

x =  [[0]]
x*2 =  [[0], [0]]
after x = x + x*2
x =  [[0], [0], [0]]
x[0] =  [0]
after x[0].append(1)
 [[0, 1], [0, 1], [0, 1]]
